In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.fft import fft2, ifft2, fftfreq
from sklearn.decomposition import PCA
from IPython.display import HTML, display
import ipywidgets as widgets

# ==========================================================
# Part A: Data generator
# ==========================================================

def fps(nx, ny, dx, dy, f):
    rhs = f[1:nx+1, 1:ny+1]
    f_hat = fft2(rhs)
    kx = fftfreq(nx, d=dx/(2*np.pi))
    ky = fftfreq(ny, d=dy/(2*np.pi))
    kx, ky = np.meshgrid(kx, ky, indexing='ij')
    ksq = kx**2 + ky**2
    ksq[0,0] = 1.0
    psi_hat = -f_hat / ksq
    psi_hat[0,0] = 0.0
    ut = np.real(ifft2(psi_hat))
    u = np.empty((nx+3, ny+3))
    u[1:nx+1, 1:ny+1] = ut
    u[:, ny+1] = u[:, 1]
    u[nx+1, :] = u[1, :]
    u[nx+1, ny+1] = u[1,1]
    return u

def bc(nx,ny,u):
    u[:,0] = u[:,ny]
    u[:,ny+2] = u[:,2]
    u[0,:] = u[nx,:]
    u[nx+2,:] = u[2,:]
    return u

def rhs(nx,ny,dx,dy,re,w,s,x,y,ts):
    aa = 1.0/(dx*dx)
    bb = 1.0/(dy*dy)
    gg = 1.0/(4.0*dx*dy)
    hh = 1.0/3.0
    f = np.zeros((nx+3,ny+3))

    j1 = gg*((w[2:nx+3,1:ny+2]-w[0:nx+1,1:ny+2])*(s[1:nx+2,2:ny+3]-s[1:nx+2,0:ny+1]) \
             -(w[1:nx+2,2:ny+3]-w[1:nx+2,0:ny+1])*(s[2:nx+3,1:ny+2]-s[0:nx+1,1:ny+2]))
    j2 = gg*( w[2:nx+3,1:ny+2]*(s[2:nx+3,2:ny+3]-s[2:nx+3,0:ny+1]) \
            - w[0:nx+1,1:ny+2]*(s[0:nx+1,2:ny+3]-s[0:nx+1,0:ny+1]) \
            - w[1:nx+2,2:ny+3]*(s[2:nx+3,2:ny+3]-s[0:nx+1,2:ny+3]) \
            + w[1:nx+2,0:ny+1]*(s[2:nx+3,0:ny+1]-s[0:nx+1,0:ny+1]))
    j3 = gg*( w[2:nx+3,2:ny+3]*(s[1:nx+2,2:ny+3]-s[2:nx+3,1:ny+2]) \
            - w[0:nx+1,0:ny+1]*(s[0:nx+1,1:ny+2]-s[1:nx+2,0:ny+1]) \
            - w[0:nx+1,2:ny+3]*(s[1:nx+2,2:ny+3]-s[0:nx+1,1:ny+2]) \
            + w[2:nx+3,0:ny+1]*(s[2:nx+3,1:ny+2]-s[1:nx+2,0:ny+1]) )
    jac = (j1+j2+j3)*hh

    lap = aa*(w[2:nx+3,1:ny+2]-2.0*w[1:nx+2,1:ny+2]+w[0:nx+1,1:ny+2]) \
        + bb*(w[1:nx+2,2:ny+3]-2.0*w[1:nx+2,1:ny+2]+w[1:nx+2,0:ny+1])

    f[1:nx+2,1:ny+2] = -jac + lap/re 
    return f

def vm_ic(nx,ny,x,y):
    w = np.empty((nx+3,ny+3))
    sigma = np.pi
    xc1, yc1 = np.pi-np.pi/4.0, np.pi
    xc2, yc2 = np.pi+np.pi/4.0, np.pi
    w[1:nx+2, 1:ny+2] = np.exp(-sigma*((x[0:nx+1, 0:ny+1]-xc1)**2 + (y[0:nx+1, 0:ny+1]-yc1)**2)) \
                       + np.exp(-sigma*((x[0:nx+1, 0:ny+1]-xc2)**2 + (y[0:nx+1, 0:ny+1]-yc2)**2))
    return bc(nx,ny,w)

def generate_vortex_data(nd=64, nt=200, re=560.0, dt=0.01, save_freq=10):
    nx, ny = nd, nd
    lx, ly = 2.0*np.pi, 2.0*np.pi
    dx, dy = lx/nx, ly/ny
    x, y = np.meshgrid(np.linspace(0.0,2.0*np.pi,nx+1),
                       np.linspace(0.0,2.0*np.pi,ny+1), indexing='ij')

    w = vm_ic(nx,ny,x,y)
    s = fps(nx, ny, dx, dy, -w); s = bc(nx,ny,s)
    t = np.empty_like(w)
    r = np.empty_like(w)

    snapshots = [np.copy(w)]
    aa, bb = 1/3, 2/3

    for k in range(1, nt+1):
        time = k*dt
        r = rhs(nx,ny,dx,dy,re,w,s,x,y,time)

        # RK3 stages
        t[1:nx+2,1:ny+2] = w[1:nx+2,1:ny+2] + dt*r[1:nx+2,1:ny+2]
        t = bc(nx,ny,t)
        s = fps(nx, ny, dx, dy, -t); s = bc(nx,ny,s)
        r = rhs(nx,ny,dx,dy,re,t,s,x,y,time)

        t[1:nx+2,1:ny+2] = 0.75*w[1:nx+2,1:ny+2] + 0.25*t[1:nx+2,1:ny+2] + 0.25*dt*r[1:nx+2,1:ny+2]
        t = bc(nx,ny,t)
        s = fps(nx, ny, dx, dy, -t); s = bc(nx,ny,s)
        r = rhs(nx,ny,dx,dy,re,t,s,x,y,time)

        w[1:nx+2,1:ny+2] = aa*w[1:nx+2,1:ny+2] + bb*t[1:nx+2,1:ny+2] + bb*dt*r[1:nx+2,1:ny+2]
        w = bc(nx,ny,w)
        s = fps(nx, ny, dx, dy, -w); s = bc(nx,ny,s)

        if k % save_freq == 0:
            snapshots.append(np.copy(w))

    snapshots = np.array(snapshots)   # (n_snaps, nx+3, ny+3)
    data = snapshots[:, 1:nx+1, 1:ny+1]   # drop ghost cells
    n_frames, nx_eff, ny_eff = data.shape
    X = data.reshape(n_frames, -1).T      # (n_pixels, n_frames)
    return data, X

# ==========================================================
# Part B: PCA with interactive controls
# ==========================================================

def run_pca_interactive(data, X):
    n_frames, nx_eff, ny_eff = data.shape

    # Full PCA
    pca = PCA()
    X_scores = pca.fit_transform(X.T)
    explained_var = np.cumsum(pca.explained_variance_ratio_)*100

    # Explained variance plot
    plt.figure(figsize=(6,4))
    plt.plot(explained_var, marker='o')
    plt.xlabel("Number of components")
    plt.ylabel("Cumulative explained variance (%)")
    plt.title("Explained Variance")
    plt.grid()
    plt.show()

    # Interactive widget
    def update(n_components=5):
        # Reduced PCA
        pca_red = PCA(n_components=n_components)
        X_scores_red = pca_red.fit_transform(X.T)
        X_recon = pca_red.inverse_transform(X_scores_red).T
        X_recon = X_recon.reshape(nx_eff, ny_eff, n_frames).transpose(2,0,1)
        error = data - X_recon

        # Memory usage
        original_mem = X.nbytes
        reduced_mem = X_scores_red.nbytes + pca_red.components_.nbytes
        compression = reduced_mem/original_mem
        mse = np.mean(error**2)

        print(f"Components: {n_components}")
        print(f"Original memory: {original_mem/1e6:.2f} MB")
        print(f"Reduced memory: {reduced_mem/1e6:.2f} MB")
        print(f"Compression ratio: {compression:.2%}")
        print(f"Reconstruction MSE: {mse:.4e}")

        # Animation
        fig, axes = plt.subplots(1, 3, figsize=(12,4))
        titles = ["Original", "Reconstructed", "Error"]

        ims = []
        for i in range(n_frames):
            im_list = []
            for ax, arr, title in zip(axes, [data, X_recon, error], titles):
                im = ax.imshow(arr[i], cmap='RdBu', animated=True)
                ax.set_title(title)
                ax.axis("off")
                im_list.append(im)
            ims.append(im_list)

        ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True)
        plt.close(fig)
        display(HTML(ani.to_jshtml()))

    return widgets.interact(update, n_components=widgets.IntSlider(min=1, max=30, step=1, value=5))

# ==========================================================
# Run everything
# ==========================================================
data, X = generate_vortex_data(nd=64, nt=600, save_freq=10)
run_pca_interactive(data, X)


In [ ]:
import pandas as pd

# ---------------------------------
# Function to add outliers
# ---------------------------------
def add_outliers(data, n_outliers, outliers_range):
    """
    Add artificial outliers into the snapshot data.
    - data: (n_frames, nx, ny) array
    - n_outliers: number of frames to corrupt
    - outliers_range: (low, high) for outlier magnitude
    """
    n_frames, nx, ny = data.shape
    df = pd.DataFrame(data.reshape(n_frames, -1))  # flatten to 2D

    # Indices of the outlier frames
    outlier_indices = np.random.choice(df.index, n_outliers, replace=False)

    for index in outlier_indices:
        col = np.random.choice(df.columns)
        if np.random.rand() > 0.5:
            df.at[index, col] = np.random.uniform(*outliers_range)
        else:
            df.at[index, col] = -np.random.uniform(*outliers_range)

    data_out = df.values.reshape(n_frames, nx, ny)
    return data_out, outlier_indices

# ---------------------------------
# Inject outliers into the data
# ---------------------------------
n_outliers = 50
outlier_range = (-50, 50)  # exaggerate for visibility
data_with_outliers, outlier_indices = add_outliers(data, n_outliers, outlier_range)

print(f"Injected outliers into frames: {outlier_indices}")

# ---------------------------------
# Redo PCA with outlier data
# ---------------------------------
X_out = data_with_outliers.reshape(data_with_outliers.shape[0], -1).T

# Re-run PCA workflow interactively
run_pca_interactive(data_with_outliers, X_out)


In [ ]:
import numpy as np

def robust_pca(M, lam=None, mu=None, max_iter=1000, tol=1e-7):
    """
    Robust PCA via Inexact Augmented Lagrange Multiplier (IALM).
    
    Decomposes M into:
        M = L + S
    where:
        L = low-rank component
        S = sparse component (outliers)
    
    Parameters
    ----------
    M : np.ndarray
        Input matrix (2D: pixels x frames).
    lam : float, optional
        Regularization parameter for sparse term. Default = 1/sqrt(max(m, n)).
    mu : float, optional
        Lagrange multiplier parameter. Default = (m*n)/(4*||M||_1).
    max_iter : int
        Maximum iterations.
    tol : float
        Convergence tolerance.
    
    Returns
    -------
    L : np.ndarray
        Low-rank matrix.
    S : np.ndarray
        Sparse matrix.
    """
    m, n = M.shape
    norm_M = np.linalg.norm(M, ord='fro')

    if lam is None:
        lam = 1 / np.sqrt(max(m, n))
    if mu is None:
        mu = (m * n) / (4.0 * np.sum(np.abs(M)))

    L = np.zeros((m, n))
    S = np.zeros((m, n))
    Y = np.zeros((m, n))

    for iter in range(max_iter):
        # Update L via singular value thresholding
        U, sigma, VT = np.linalg.svd(M - S + (1/mu)*Y, full_matrices=False)
        sigma_thresh = np.maximum(sigma - 1/mu, 0)
        L = U @ np.diag(sigma_thresh) @ VT

        # Update S via shrinkage
        residual = M - L + (1/mu)*Y
        S = np.sign(residual) * np.maximum(np.abs(residual) - lam/mu, 0)

        # Update multiplier
        Z = M - L - S
        Y = Y + mu * Z

        err = np.linalg.norm(Z, ord='fro') / norm_M
        if err < tol:
            break

    return L, S

# Convert frames into matrix (pixels x frames)
M = data_with_outliers.reshape(data_with_outliers.shape[0], -1).T  

# Apply RPCA
L, S = robust_pca(M)

# Convert back to frames
n_frames, nx, ny = data_with_outliers.shape
L_frames = L.T.reshape(n_frames, nx, ny)
S_frames = S.T.reshape(n_frames, nx, ny)

# Animate
fig, axes = plt.subplots(1, 3, figsize=(12,4))
titles = ["Original with Outliers", "Low-Rank (Clean)", "Sparse (Outliers)"]

ims = []
for i in range(n_frames):
    im_list = []
    for ax, arr, title in zip(axes, [data_with_outliers, L_frames, S_frames], titles):
        im = ax.imshow(arr[i], cmap='RdBu', animated=True)
        ax.set_title(title)
        ax.axis("off")
        im_list.append(im)
    ims.append(im_list)

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True)
plt.close(fig)
display(HTML(ani.to_jshtml()))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.fft import fft2, ifft2, fftfreq
from sklearn.decomposition import PCA
from IPython.display import HTML, display
import torch

# ==========================================================
# Device setup (CPU / GPU)
# ==========================================================
device = (
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(f"Using device: {device}")

# ==========================================================
# Part A: Data generator (same vortex merger solver)
# ==========================================================
def fps(nx, ny, dx, dy, f):
    rhs = f[1:nx+1, 1:ny+1]
    f_hat = fft2(rhs)
    kx = fftfreq(nx, d=dx/(2*np.pi))
    ky = fftfreq(ny, d=dy/(2*np.pi))
    kx, ky = np.meshgrid(kx, ky, indexing='ij')
    ksq = kx**2 + ky**2
    ksq[0,0] = 1.0
    psi_hat = -f_hat / ksq
    psi_hat[0,0] = 0.0
    ut = np.real(ifft2(psi_hat))
    u = np.empty((nx+3, ny+3))
    u[1:nx+1, 1:ny+1] = ut
    u[:, ny+1] = u[:, 1]
    u[nx+1, :] = u[1, :]
    u[nx+1, ny+1] = u[1,1]
    return u

def bc(nx,ny,u):
    u[:,0] = u[:,ny]
    u[:,ny+2] = u[:,2]
    u[0,:] = u[nx,:]
    u[nx+2,:] = u[2,:]
    return u

def rhs(nx,ny,dx,dy,re,w,s,x,y,ts):
    aa, bb, gg, hh = 1/(dx*dx), 1/(dy*dy), 1/(4*dx*dy), 1/3
    f = np.zeros((nx+3,ny+3))

    j1 = gg*((w[2:nx+3,1:ny+2]-w[0:nx+1,1:ny+2])*(s[1:nx+2,2:ny+3]-s[1:nx+2,0:ny+1])
             -(w[1:nx+2,2:ny+3]-w[1:nx+2,0:ny+1])*(s[2:nx+3,1:ny+2]-s[0:nx+1,1:ny+2]))
    j2 = gg*( w[2:nx+3,1:ny+2]*(s[2:nx+3,2:ny+3]-s[2:nx+3,0:ny+1])
            - w[0:nx+1,1:ny+2]*(s[0:nx+1,2:ny+3]-s[0:nx+1,0:ny+1])
            - w[1:nx+2,2:ny+3]*(s[2:nx+3,2:ny+3]-s[0:nx+1,2:ny+3])
            + w[1:nx+2,0:ny+1]*(s[2:nx+3,0:ny+1]-s[0:nx+1,0:ny+1]))
    j3 = gg*( w[2:nx+3,2:ny+3]*(s[1:nx+2,2:ny+3]-s[2:nx+3,1:ny+2])
            - w[0:nx+1,0:ny+1]*(s[0:nx+1,1:ny+2]-s[1:nx+2,0:ny+1])
            - w[0:nx+1,2:ny+3]*(s[1:nx+2,2:ny+3]-s[0:nx+1,1:ny+2])
            + w[2:nx+3,0:ny+1]*(s[2:nx+3,1:ny+2]-s[1:nx+2,0:ny+1]) )
    jac = (j1+j2+j3)*hh
    lap = aa*(w[2:nx+3,1:ny+2]-2*w[1:nx+2,1:ny+2]+w[0:nx+1,1:ny+2]) \
        + bb*(w[1:nx+2,2:ny+3]-2*w[1:nx+2,1:ny+2]+w[1:nx+2,0:ny+1])
    f[1:nx+2,1:ny+2] = -jac + lap/re 
    return f

def vm_ic(nx,ny,x,y):
    w = np.empty((nx+3,ny+3))
    sigma = np.pi
    xc1, yc1 = np.pi-np.pi/4.0, np.pi
    xc2, yc2 = np.pi+np.pi/4.0, np.pi
    w[1:nx+2, 1:ny+2] = np.exp(-sigma*((x[0:nx+1,0:ny+1]-xc1)**2+(y[0:nx+1,0:ny+1]-yc1)**2)) \
                       + np.exp(-sigma*((x[0:nx+1,0:ny+1]-xc2)**2+(y[0:nx+1,0:ny+1]-yc2)**2))
    return bc(nx,ny,w)

def generate_vortex_data(nd=64, nt=200, re=560.0, dt=0.01, save_freq=10):
    nx, ny = nd, nd
    lx, ly = 2*np.pi, 2*np.pi
    dx, dy = lx/nx, ly/ny
    x, y = np.meshgrid(np.linspace(0,2*np.pi,nx+1),
                       np.linspace(0,2*np.pi,ny+1), indexing='ij')

    w = vm_ic(nx,ny,x,y)
    s = fps(nx,ny,dx,dy,-w); s=bc(nx,ny,s)
    t, r = np.empty_like(w), np.empty_like(w)
    snapshots=[np.copy(w)]
    aa, bb=1/3,2/3

    for k in range(1, nt+1):
        r = rhs(nx,ny,dx,dy,re,w,s,x,y,k*dt)
        t[1:nx+2,1:ny+2]=w[1:nx+2,1:ny+2]+dt*r[1:nx+2,1:ny+2]; t=bc(nx,ny,t)
        s=fps(nx,ny,dx,dy,-t); s=bc(nx,ny,s)
        r=rhs(nx,ny,dx,dy,re,t,s,x,y,k*dt)

        t[1:nx+2,1:ny+2]=0.75*w[1:nx+2,1:ny+2]+0.25*t[1:nx+2,1:ny+2]+0.25*dt*r[1:nx+2,1:ny+2]; t=bc(nx,ny,t)
        s=fps(nx,ny,dx,dy,-t); s=bc(nx,ny,s)
        r=rhs(nx,ny,dx,dy,re,t,s,x,y,k*dt)

        w[1:nx+2,1:ny+2]=aa*w[1:nx+2,1:ny+2]+bb*t[1:nx+2,1:ny+2]+bb*dt*r[1:nx+2,1:ny+2]; w=bc(nx,ny,w)
        s=fps(nx,ny,dx,dy,-w); s=bc(nx,ny,s)
        if k%save_freq==0: snapshots.append(np.copy(w))

    data=np.array(snapshots)[:,1:nx+1,1:ny+1] # drop ghost cells
    n_frames,nx_eff,ny_eff=data.shape
    X=data.reshape(n_frames,-1).T
    return data,X

# ==========================================================
# PCA utilities
# ==========================================================
def run_pca(data,X,n_components=10,title="Clean Data"):
    n_frames,nx,ny=data.shape
    X_t=torch.tensor(X.T,dtype=torch.float32,device=device)
    U,S,V=torch.pca_lowrank(X_t,q=min(n_components+5,X_t.shape[1]))
    scores=torch.matmul(X_t,U[:,:n_components])
    recon=torch.matmul(scores,U[:,:n_components].T).cpu().numpy().T
    recon=recon.reshape(nx,ny,n_frames).transpose(2,0,1)
    error=data-recon

    # Explained variance
    var=(S**2)/torch.sum(S**2)
    cum_var=torch.cumsum(var,0).cpu().numpy()*100
    plt.figure(figsize=(6,4))
    plt.plot(cum_var,marker='o'); plt.title(f"Explained Variance ({title})")
    plt.xlabel("Components"); plt.ylabel("Cumulative %"); plt.grid(); plt.show()

    # First 10 modes
    fig,axes=plt.subplots(2,5,figsize=(15,6))
    for i,ax in enumerate(axes.flat[:10]):
        mode=U[:,i].cpu().numpy().reshape(nx,ny)
        ax.imshow(mode,cmap="RdBu"); ax.set_title(f"Mode {i+1}"); ax.axis("off")
    plt.suptitle(f"First 10 PCA Modes ({title})"); plt.show()

    # Animation
    fig,axes=plt.subplots(1,3,figsize=(12,4))
    titles=["Original","Reconstruction","Error"]; ims=[]
    for i in range(n_frames):
        im_list=[]
        for ax,arr,t in zip(axes,[data,recon,error],titles):
            im=ax.imshow(arr[i],cmap="RdBu",animated=True); ax.set_title(t); ax.axis("off")
            im_list.append(im)
        ims.append(im_list)
    ani=animation.ArtistAnimation(fig,ims,interval=100,blit=True); plt.close(fig)
    display(HTML(ani.to_jshtml()))

    return recon,error

# ==========================================================
# Outlier injection
# ==========================================================
def add_outliers(data,n_outliers,low=-50,high=50):
    n_frames,nx,ny=data.shape
    data_noisy=data.copy()
    indices=[]
    for _ in range(n_outliers):
        f=np.random.randint(0,n_frames)
        i,j=np.random.randint(0,nx),np.random.randint(0,ny)
        data_noisy[f,i,j]=np.random.uniform(low,high)
        indices.append((f,i,j))
    return data_noisy,indices

# ==========================================================
# Run pipeline
# ==========================================================
data,X=generate_vortex_data(nd=64,nt=600,save_freq=10)
print("=== PCA on Clean Data ===")
recon_clean,_=run_pca(data,X,n_components=10,title="Clean")

# Add outliers
data_noisy,out_idx=add_outliers(data,n_outliers=30)
print(f"Outliers injected at: {out_idx[:10]} ... (total {len(out_idx)})")

X_noisy=data_noisy.reshape(data_noisy.shape[0],-1).T
print("=== PCA on Noisy Data ===")
recon_noisy,_=run_pca(data_noisy,X_noisy,n_components=10,title="With Outliers")
